# Setup

In [1]:
%%capture
!pip install eikon
!pip install more_itertools

In [2]:
import eikon as ek
import pandas as pd
import numpy as np
import time
import os
import glob
from tqdm.auto import tqdm
from more_itertools import chunked
from math import ceil, floor
from datetime import datetime

In [3]:
#"api_key_lk"
ek.set_app_key("716a396b553b441680427f612d2f89735c88bf9e")

# Interesting Eikon Data Items

The below is a collection of some Data Items from the Eikon platform we've found useful. Also refer to Fosse (2022) for more items regarding bonds.

In [7]:
#Market capitalization fields 
fields = ['TR.CompanyMarketCap.Date','TR.CompanyMarketCap', 'TR.PriceClose', 'TR.CompanyMarketCap.Currency'] 

#Fundamental stock data fields
profits = ['TR.TotalRevenue', 'TR.GrossProfit','TR.EBITDA','TR.EBIT', 'TR.F.NetIncAfterTax']#, 'TR.EV','MKT_CAP']
balance = ['TR.F.TotAssets','TR.F.TotCurrAssets','TR.F.TotLiab','TR.F.TotCurrLiab', 'TR.F.LTDebtPctofTotAssets','TR.F.STDebtPctofTotAssets',"TR.InvtrPriceToBook"]#TR.F.TotLiab(Period=FY0)
cash_flow = ['TR.F.LeveredFOCF']

fundamental_data = profits + balance + cash_flow 
reported_dates = ['TR.TotalRevenue.periodenddate']

#Company meta data fields
geography = ['TR.ExchangeMarketIdCode', 'TR.HeadquartersRegionAlt', 'TR.HeadquartersCountry', 'TR.HQStateProvince']
sectors = ['TR.TRBCEconomicSector', 'TR.TRBCBusinessSector', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustry', 'TR.TRBCActivity']
founded = ['TR.OrgFoundedYear']

meta_data = geography + founded + sectors

#Broker estimates
#params_new["Period"] = "FY1"    

fields = ["TR.EPSMean","TR.EPSMean.periodenddate","TR.EBITMean",'TR.RevenueMean',
              "TR.ROAMean","TR.ROEMean","TR.FCFMean","TR.TotalAssets","TR.MeanPctChg(Period=FY1,WP=60d)"]

# Collect data from eikon refinitiv

In [121]:
def get_timeseries_wrapped(tickers, fields, params):
    df = ek.get_timeseries(
        tickers,
        fields,
        start_date=params.get("SDate"),
        end_date=params.get("EDate"),
        interval=params.get("Frq", "daily")
    )
    
    df["ticker"] = tickers[0]
    
    return df, None

In [9]:
def _decide_get_function(fields, params):
    
    timeseries_fields = ["TIMESTAMP", "VALUE", "VOLUME", "HIGH", "LOW", "OPEN", "CLOSE", "COUNT"]
    
    if isinstance(fields, str):
        fields = [fields]
    
    if fields is None or all([field in timeseries_fields for field in fields]): 
        return get_timeseries_wrapped
    
    if "Frq" in params.keys():
        if params["Frq"] in ["tick", "minute", "hour"]:
                raise ValueError("You requested data of ek.get_data() requesting timeperiods it does not support")
        
    return ek.get_data

In [10]:
def correct_fields(fields):
    if fields is None or len(fields) == 0:
        return None
    return fields

In [11]:
def correct_tickers(tickers):
    if isinstance(tickers, str):
        return [tickers]
    return tickers

In [12]:
def correct_params(chosen_function, params):
    correction = {"start_date": "SDate", "end_date":"EDate", "interval":"Frq"}
    if chosen_function == ek.get_timeseries:
        for correction_name, actual_name in correction.items():
            if actual_name in params:
                params[correction_name] = params[actual_name]
            

In [17]:
def handle_and_update_server_side_problem(server_side_problems, err):
    server_side_problems += 1
    print(err.message)
    seconds_to_wait = wait_time(server_side_problems)
    print(f"waiting {seconds_to_wait} second(s) before new pull request...")
    time.sleep(wait_time(seconds_to_wait))

In [13]:
def _get_many_params(params, date_range, chosen_function):
    if date_range is not None:
        params_copies = []       
        for i in range(len(date_range)-1):
            new_params = params.copy()
            new_params["SDate"] = date_range[i].strftime('%Y-%m-%d')
            new_params["EDate"] = date_range[i+1].strftime('%Y-%m-%d')
            correct_params(new_params, chosen_function)
            params_copies.append(new_params)
        return params_copies
    return [params]

In [91]:
def register_error(d, tickers, err):
    for ticker in tickers:
        d[ticker] = err        

In [114]:
def get_data_and_handle_errors(getting_function, tickers, fields, params):
    ticker_and_error = {}
    
    server_side_problems = 0
    
    while server_side_problems < 5:
        
        try:
            #if there is only one ticker causing the error should get_and_log perhaps split call into separate tickers?
            #fields not avalible should (and I think are) set to NA 
            new_df, err = getting_function(tickers, fields, params)
           
            return new_df, pd.DataFrame.from_dict({}, columns=["error"], orient="index")
        
        except ek.EikonError as err:
            print("In except, tickers", tickers)
            register_error(ticker_and_error, tickers, err)
            if err.code == -1:
                # Data missing or something else bad
                print("Data missing for a ticker")
                return None, pd.DataFrame.from_dict(ticker_and_error, columns=["error"], orient="index")

            if err.code == 408:
                # HTTP Timeout exception, this just happens frequently... think it can be server side problem
                handle_and_update_server_side_problem(server_side_problems, err)
            
            if err.code == 400 or err.code == 2504: # Backend error, Eikon suggests waiting this one out 
                handle_and_update_server_side_problem(server_side_problems, err)
            
            if err.code == 429:
                print(err.message)
                print("If this error message keeps happening, your subscription has probably collected too much data today")
                # Either daily limit is reached or call based limit reached latter can be a problem from get_datas
    
    # Can only end up here if the while loops breaks without having returned --> there is an error
    return None, pd.DataFrame.from_dict(ticker_and_error, columns=["error"], orient="index")

In [105]:
def _decide_tickers_and_params(chosen_function, fields, lst_of_tickers, params):
    
    data_limit = {get_timeseries_wrapped: 2_500, ek.get_data: 9_000}
    date_range = None
    tickers_params_pairs = []
    
    # If there this is not a timeseries problem
    if "SDate" not in params.keys():
        return [(lst_of_tickers, params)]
    
    for ticker in lst_of_tickers:
        df, err = get_data_and_handle_errors(chosen_function, ticker, fields if fields is None else fields[0], params)
        if df is not None: break
    else: raise ValueError("Not able to get data for any of the chosen tickers")
    
    fields = ["index"] + df.columns.to_list()

    number_of_timepoints = df.shape[0]
    print(f"[INFO] Number of dates: {number_of_timepoints}")
    
    number_of_tickers_at_once = floor(data_limit[chosen_function] / (len(fields) * number_of_timepoints))
    print(f"[INFO] Number of tickers at once: {min(max(number_of_tickers_at_once, 1), len(lst_of_tickers))}")
    
    if number_of_tickers_at_once < 1: 
        number_of_timepoints_at_once = floor(data_limit[chosen_function] / len(fields))
        print(f"[INFO] Max time window length: {number_of_timepoints_at_once}")
        intervals_needed = ceil(number_of_timepoints / number_of_timepoints_at_once)
        
        time_delta = (pd.to_datetime(params["EDate"]) - pd.to_datetime(params["SDate"])) / intervals_needed
        date_range = [pd.to_datetime(params["SDate"]) + i*time_delta for i in range(intervals_needed-1)] + [pd.to_datetime(params["EDate"])]
        number_of_tickers_at_once = 1
    
    if chosen_function is get_timeseries_wrapped:
        number_of_tickers_at_once = 1
    
    sub_ticker_lst = list(chunked(lst_of_tickers, number_of_tickers_at_once))
    params_copies = _get_many_params(params, date_range, chosen_function)
    
    for sub_tickers in sub_ticker_lst:
        for params_copy in params_copies:
            tickers_params_pairs.append((sub_tickers, params_copy))
            
    
    return tickers_params_pairs

In [15]:
#Function to maximize data per http request. 
def _divide_pull_request(lst_of_tickers, fields, params):
    #batch, deduce get_data or get_timeseries size given
    #lst_of_tickers x fields x (params[start]-params[end])*freq < 10,000 or 3,000
    #also adjust params so that they fit selected function 
    tickers = correct_tickers(lst_of_tickers)
    chosen_function = _decide_get_function(fields, params)
    fields = correct_fields(fields)
    correct_params(chosen_function, params)
    tickers_params_pairs = _decide_tickers_and_params(chosen_function, fields, tickers, params)#TODO
    
    return tickers_params_pairs, chosen_function, fields

In [16]:
def wait_time(server_side_problems):
    return 3**(server_side_problems-1)

In [19]:
def save_if_criteria_met(df, filename, count):
    if (not count % 10 or count == -1) and filename is not None:
        df.to_csv(filename)

In [20]:
def format_filename(filename):
    if filename is None:
        return None
    
    if filename[-4:] == ".csv":
        return filename
    return filename + ".csv"

def format_error_filename(filename):
    if filename is None:
        return None

    if filename[-4:] == ".csv":
        return filename[:-4] + "_errors.csv"
    return filename + "_errors.csv"

In [128]:
def df_append(old, new):
    if old is None:
        return new
    
    if new is None:
        return old

    return pd.concat([old, new])

In [129]:
def get_data(lst_of_tickers, fields, params, filename=None):
    # (return or save) requested data and (return or save) non-retreived-data 
    
    saved_df = None
    error_df = None
    
    filename = format_filename(filename)
    error_filename = format_error_filename(filename)
    
    tickers_params_pairs, chosen_function, fields = _divide_pull_request(lst_of_tickers, fields, params)
    
    print("Starting for loop")
    for i, (tickers, param) in enumerate(tqdm(tickers_params_pairs, desc="Ticker")):  
        
        new_df, ticker_and_error = get_data_and_handle_errors(chosen_function, tickers, fields, param)
        
        # Merge new data with the one we've already fetched
        saved_df = df_append(saved_df, new_df)
        error_df = df_append(error_df, ticker_and_error)
        
        # In case user has requested writing to file -> log result as it stands
        save_if_criteria_met(saved_df, filename, i)
        save_if_criteria_met(error_df, error_filename, i)
    
    # In case user has requested writing to file -> log final df result
    save_if_criteria_met(saved_df, filename, -1)
    save_if_criteria_met(error_df, error_filename, -1)
    
    return saved_df, error_df

# Examples

## Example 0) Screen companies

Below, we've created a couple of simple screens we used outselves in our Master's thesis. The first one gets all Norwegian oil-related companies, the second gets all global oil-related companies, and the last gets all global financial companies (all with some extra criterias).

In [28]:
# For all these screens, we're only interested in the names of the companies
# (to inspect what companies come out)
company_names = ["TR.CommonName"]

In [29]:
# Norwegian oil companies
# First create the screening string (see the readme for more on this)
oil_osebx_screen = 'SCREEN(U(IN(Equity(active,public,primary))),' \
    'TR.CompanyMarketCap>=500000, IN(TR.ExchangeMarketIdCode,"XOSL"),'\
    'IN(TR.TRBCBusinessSectorCode,"5010","5020","5030"), CURN=USD)'

# Get companies in screen and show the first 5
osbx_companies, e = get_data(oil_osebx_screen, company_names, {})
osbx_companies = osbx_companies.set_index("Instrument"); print(f"Number of companies: {osbx_companies.shape[0]}")
osbx_companies.head(5)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of companies: 46


,Company Common Name
Instrument,
BONHR.OL,Bonheur ASA
DNO.OL,Dno ASA
PSE.OL,Petrolia E&P Holdings Plc
MSEIS.OL,Magseis Fairfield ASA
ODF.OL,Odfjell SE


In [56]:
# Global oil companies
oil_global_screen = 'SCREEN(U(IN(Equity(active,public,primary))),'\
    'TR.CompanyMarketCap>=500000, IN(TR.TRBCBusinessSectorCode,"5010","5020","5030"), CURN=USD)'


# Get and show data
global_oil, e = get_data(oil_global_screen, company_names, {})
global_oil = global_oil.set_index("Instrument"); print(f"Number of companies: {global_oil.shape[0]}")
global_oil.head(5)

Ticker:   0%|          | 0/1 [00:00<?, ?it/s]

Number of companies: 2029


,Company Common Name
Instrument,
OMVV.VI,OMV AG
IMO.TO,Imperial Oil Ltd
ENB.TO,Enbridge Inc
REP.MC,Repsol SA
DPAP.PA,Les Docks des Petroles d Ambes SA


In [31]:
# Global financial companies
financial_global_screen = 'SCREEN(U(IN(Equity(active,public,primary))),'\
    'TR.CompanyMarketCap>=500000, IN(TR.TRBCBusinessSectorCode,"5510"), CURN=USD)'

global_financial, e = ek.get_data(financial_global_screen, company_names, {})
global_financial = global_financial.set_index("Instrument"); print(f"Number of companies: {global_financial.shape[0]}")
global_financial.head(5)

Number of companies: 4601


,Company Common Name
Instrument,
KAER.VI,BKS Bank AG
TIRO.VI,Bank fuer Tirol und Vorarlberg AG
000416.SZ,Minsheng Holdings Co Ltd
NEDJ.J,Nedbank Group Ltd
SBKJ.J,Standard Bank Group Ltd


## Example 1) Stock and fundamental data retrival

### Founded year for oil companies

To get started, we'll start with a simple example where we get the years the global oil companies were founded. In short, we first create a list of tickers from the screens above. Then, we define what fields we're interested in, then we fetch the data.

Please note in the data frame result that `PUMA.LZ` do not have a year. Already, we see that Eikon suffers from faulty or missing data. Please use this example as a warning to always inspect the data you get from Eikon because there can always be faults with the data from Eikon's side.

In [22]:
# Tickers to get
lst_of_tickers = global_oil.index.to_list()

# Fields we want data for for the above tickers (see Data Item Browser or our/Fosse (2022)'s list of common fields)
fields = ['TR.OrgFoundedYear']

# Eikon parameters (none required as we're getting static data)
ek_params = {}

df, err = get_data(lst_of_tickers, fields, ek_params)
df

  0%|          | 0/1 [00:00<?, ?it/s]

,Instrument,Organization Founded Year
0,OMVV.VI,1956
1,IMO.TO,1978
2,ENB.TO,1987
3,REP.MC,1987
4,DPAP.PA,1976
...,...,...
2024,PUMA.LZ,0
2025,PGS.HN,2007
2026,PVG.HN,2007
2027,TPL.CN,2001


### Fundamental and stock price data

In this example, we are interested to find Norwegian publicly traded companys fundamental data quarterly fundamental data, and its stock price, in daily format. 

To begin, we use a single ticker in the oil space, AKRBP.OL (which really did a "kule" during the spring of 2022). We chose this ticker at random from the above screening result (`global_oil`). We are interested in its stock price possibly from 2020 and will just use `get_data` and hope that it hands us some interesting data without asking for specific fields. Surely, if asking for a stock, with no more information, `get_data` would understand that we are interested in stock price data. 

In [53]:
ticker = "AKRBP.OL"
start_date = "2020-01-01"
end_date = "2022-05-06" # A very nice man's birthday (according to some)
fields = None

params = {"SDate": start_date, "EDate": end_date}

df, err = get_data(ticker, fields, params)
df

[INFO] Number of dates: 591
[INFO] Number of tickers at once: 1
[INFO] Max time window length: 416


Ticker:   0%|          | 0/1 [00:00<?, ?it/s]

AKRBP.OL,HIGH,CLOSE,LOW,OPEN,VOLUME
Date,,,,,
2020-01-02,293.3,289.0,287.6,288.1,580227
2020-01-03,295.8,292.9,291.4,293.6,553248
2020-01-06,297.7,296.3,292.6,297.7,513372
2020-01-07,297.1,295.7,293.0,294.9,618715
2020-01-08,297.9,295.3,294.4,297.5,672176
...,...,...,...,...,...
2022-05-02,339.7,332.2,323.4,339.7,808277
2022-05-03,343.2,343.2,323.5,331.0,771037
2022-05-04,352.1,351.0,340.9,345.0,699762


Wouldn't you know it. The `get_data` function gives us the daily stock data also with intraday Volume. 

Now, we are interested in how the financials of AKER.BP has been during the same time window. IMPORTANT NOTE ABOUT FREQUENCY:

> Note that we below use frequency AND period the iterate through time on a fical quarter (`FQ`) manner, and returning the quarterly data at that time point (`FQ0`). If we do not specify period, `get_data` will return 4 copies of the yearly financial, one for each quarter. NOTE THIS.

We use the above cell with interesting fields for inspiration for what data items we could use. We, of course, could also use the Eikon Data Item Browser.

And for good measure, let's turn the dial to 11 and get all of `AKER.BP`'s annual reports since the year 2000!

In [54]:
ticker = "AKRBP.OL"

start_date = '2000-01-01'
end_date = '2022-04-21'
frequency = "FQ"  # <-- We understand this intuitivly (refer to above note for more detail on this)
period = "FQ0"  # <-- However, this is very important to note, needed to overwrite Eikon's
                # standard of returning yearly data regardless of sampling frequency
curn = "USD"  # <-- What on earth could this possibly do?!?! ;)

date = ['TR.TotalRevenue.date']  # <-- Pro tip: use the date atrtibute of one of the fields included in the search
profit = ['TR.TotalRevenue', 'TR.GrossProfit','TR.EBITDA','TR.EBIT', 'TR.F.NetIncAfterTax']
balance = ['TR.F.TotAssets', 'TR.F.TotLiab']

fields = date + profit + balance
ek_params = {"SDate": start_date, "EDate": end_date, "Frq":frequency, "period":period, "curn":curn}
             
df, err = get_data(ticker, fields, ek_params)
df

[INFO] Number of dates: 63
[INFO] Number of tickers at once: 1


Ticker:   0%|          | 0/1 [00:00<?, ?it/s]

,Instrument,Date,Total Revenue,Gross Profit,EBITDA,EBIT,Net Income after Tax,Total Assets,Total Liabilities
0,AKRBP.OL,2006-06-30T00:00:00Z,0,0,0.0,0.0,320.81118,<NA>,<NA>
1,AKRBP.OL,2006-09-30T00:00:00Z,0,-171542,-2199210.998,-2199210.998,-498265.76248,2841282.7061,3598550.30355
2,AKRBP.OL,2006-12-31T00:00:00Z,0,-798794,-3575630.826,-3575630.826,-776947.04614,233608811.44911,77295116.15967
3,AKRBP.OL,2007-03-31T00:00:00Z,0,-541811,-3304305.06,-3304305.06,-951208.40056,220798381.52571,61525708.0825
4,AKRBP.OL,2007-06-30T00:00:00Z,0,-952083,-36077349.578,-36205953.274,-5792294.15342,234434930.34319,75832144.07068
...,...,...,...,...,...,...,...,...,...
58,AKRBP.OL,2020-12-31T00:00:00Z,833508000,691440000,622691000.0,333283000.0,129467000.0,12420091000.0,10432810000.0
59,AKRBP.OL,2021-03-31T00:00:00Z,1133238000,957332000,878190000.0,620636000.0,127030000.0,12241198000.0,10252205000.0
60,AKRBP.OL,2021-06-30T00:00:00Z,1123754000,965519000,854535000.0,614162000.0,153811000.0,13075850000.0,11045546000.0
61,AKRBP.OL,2021-09-30T00:00:00Z,1562675000,1353877000,1249866000.0,1003019000.0,205834000.0,13582017000.0,11454157000.0


This is pretty fantastic! Just contemplate what you just did. That's 63 quarterly reports in one easy swoooosh. And with this our first example is complete. Again, please note that, e.g., `Total Revenue` is 0 for several entries here, again because Eikon is flawed. Another thing, Aker BP is the result of several mergers throughout its history (most notably the merger with BP Norway in 2016), so the financial data might behave strangely at times, so use your judgement. Always check your data!

### The grand finale

Finally, let's pump this up to get this data for all oil companies in the world. See how we just change the list of tickers we're interested in and keep all other parameters constant (that's orthogonal programming for you, dear sir).

In [57]:
df, err = get_data(global_oil.index.to_list(), fields, ek_params)
df

[INFO] Number of dates: 88
[INFO] Number of tickers at once: 34


Ticker:   0%|          | 0/60 [00:00<?, ?it/s]

,Instrument,Date,Total Revenue,Gross Profit,EBITDA,EBIT,Net Income after Tax,Total Assets,Total Liabilities
0,OMVV.VI,2000-03-31T00:00:00Z,1588821544.31517,175011232.94743,152552986.052025,78828523.082512,<NA>,<NA>,<NA>
1,OMVV.VI,2000-06-30T00:00:00Z,1591395023.28372,230190745.555143,181045434.200878,105530002.190288,<NA>,<NA>,<NA>
2,OMVV.VI,2000-09-30T00:00:00Z,1824306960.87875,260038352.43591,193853005.894257,138474386.051732,<NA>,<NA>,<NA>
3,OMVV.VI,2000-12-31T00:00:00Z,1938098647.94837,372273990.672257,228378009.139304,132398360.578508,<NA>,<NA>,<NA>
4,OMVV.VI,2001-03-31T00:00:00Z,1675754784.0278,270127135.374168,246253937.336036,174937923.894256,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
384,CSIQ.OQ,2020-12-31T00:00:00Z,1040654000.0,141088000.0,62598000.0,2480000.0,3799000.0,6536854000.0,4644069000.0
385,CSIQ.OQ,2021-03-31T00:00:00Z,1089339000.0,194602000.0,105483000.0,43483000.0,12393000.0,6839473000.0,4964898000.0
386,CSIQ.OQ,2021-06-30T00:00:00Z,1429661000.0,184787000.0,92380000.0,26380000.0,17955000.0,7024607000.0,5060348000.0
387,CSIQ.OQ,2021-09-30T00:00:00Z,1229450000.0,228629000.0,124093000.0,53093000.0,34307000.0,7481829000.0,5448107000.0


Unbelievable. 87 500 quartely reports, sheeesh. And it only took 5-6 minutes on our super-slow internet connnection here we sit in Zambia making this tool for you, dear student at NTNU.

## Example 2) Bond data

In this example, we want to collect data for a series of bonds. We want to see what they are trading at, and find other interesting information like their maturity, and principal. We'll start with getting the historical prices for a specific bond we're interested in. This will, of course, scale to more by just looping over all the bond `RIC`s you're interested in. I'd like to see you `for` loop in Excel.

### Historical bond price data

The below ticker we've gotten from Eikon's Corporate Bond Advanced Search. Please see Fosse (2021) for more on how this tool works.

In [130]:
# tickers = "NZNFF1111="

curn = "USD"

start_date = '2010-01-01'
end_date = datetime.now().strftime("%Y-%m-%d")

fields = []  # <-- We just want the price data so no need to specify fields

ek_params = {"SDate": start_date, "EDate": end_date, "curn": curn}

# Call Eikon server and show result
df, err = get_data(tickers[:5], fields, ek_params)
df

2022-08-08 16:46:46,466 P[18908] [MainThread 13432] Error with NO0012555558=OL: No data available for the requested date range
2022-08-08 16:46:46,466 P[18908] [MainThread 13432] NO0012555558=OL: No data available for the requested date range | 


In except, tickers NO0012555558=OL
Data missing for a ticker
[INFO] Number of dates: 38
[INFO] Number of tickers at once: 5
Starting for loop


Ticker:   0%|          | 0/5 [00:00<?, ?it/s]

2022-08-08 16:46:47,666 P[18908] [MainThread 13432] Error with NO0012555558=OL: No data available for the requested date range
2022-08-08 16:46:47,674 P[18908] [MainThread 13432] NO0012555558=OL: No data available for the requested date range | 


In except, tickers ['NO0012555558=OL']
Data missing for a ticker


2022-08-08 16:46:48,602 P[18908] [MainThread 13432] Error with XS1639098398=TE: No data available for the requested date range
2022-08-08 16:46:48,602 P[18908] [MainThread 13432] XS1639098398=TE: No data available for the requested date range | 


In except, tickers ['XS1639098398=TE']
Data missing for a ticker


,HIGH,CLOSE,LOW,OPEN,COUNT,ticker
Date,,,,,,
2022-06-09,8.301,8.301,8.301,8.301,-1,GB248686359=
2022-06-10,8.608,8.301,8.301,8.301,-1,GB248686359=
2022-06-13,8.735,8.735,8.672,8.672,-1,GB248686359=
2022-06-21,9.493,9.361,8.984,9.062,-1,GB248686359=
2022-06-22,9.351,9.177,9.021,9.269,-1,GB248686359=
...,...,...,...,...,...,...
2022-08-02,99.92,99.92,99.92,99.92,-1,NO1080260=
2022-08-03,99.92,99.92,99.92,99.92,-1,NO1080260=
2022-08-04,99.93,99.93,99.93,99.93,-1,NO1080260=


In [131]:
err

,error
NO0012555558=OL,Error code -1 | NO0012555558=OL: No data avail...
XS1639098398=TE,Error code -1 | XS1639098398=TE: No data avail...


### Mass download of bond data

Here we'll try to get a lot of data about a large number of bonds. The list of bonds we're using below comes from the Advanced Search feature in Eikon.

In [68]:
tickers = pd.read_excel("data/bond_tickers.xlsx")["Preferred RIC"].to_list()
tickers[:5]

['NO0012555558=OL',
 'GB248686359=',
 'XS1639098398=TE',
 'GB165801202=',
 'NO1080260=']

In [66]:
fields = [
    "TR.FiMaturityStandardYield",
    "TR.IssuerRating",
    "TR.IR.RatingDate",
    "TR.CouponRate",
    "TR.CA.AmtOutstanding",
]

ek_params = {"curn": curn}

# Call Eikon server and show result
df, err = get_data(tickers, fields, ek_params)
df

Ticker:   0%|          | 0/1 [00:00<?, ?it/s]

,Instrument,Yield To Maturity,Issuer Rating,Date,Coupon Rate,Amount Outstanding
0,NO0012555558=OL,<NA>,A2,2018-09-04T00:00:00Z,5.19,100000000.0
1,GB248686359=,8.445017,Baa2,2022-06-27T00:00:00Z,8.25,350000000.0
2,XS1639098398=TE,<NA>,A1,2022-04-27T00:00:00Z,5.879,200000000.0
3,GB165801202=,7.483742,BBB,2022-07-19T00:00:00Z,5.875,1250000000.0
4,NO1080260=,5.453575,A2,2018-09-04T00:00:00Z,4.52,66000000.0
...,...,...,...,...,...,...
995,XS1649337174=LU,<NA>,A2,2022-06-30T00:00:00Z,10.8,9000000.0
996,AT0000A1X3G7=TEMK,<NA>,P-1,2018-06-08T00:00:00Z,3.75,3450000.0
997,CA064151XP3=,<NA>,WR,2021-09-03T00:00:00Z,0.0,<NA>
998,XS1627839472=GSIL,<NA>,A1,2020-10-29T00:00:00Z,5.0,100000000.0


## Example 3) Macro data

In this last example, we want to collect data on some interesting macro economic varibales. We want the oil price, which could feed into example 1) and we want the interest rate in the US, which is intersting given example 2) 

This time, we start by using the eikon refinitiv desktop search tool to look up the tickers for BRT- and the US 10y rate. Again, we use the get_data function, which will under the hood use the right functionality (ek.get_timeseries) but we do not have to think about that!

In [132]:
start_date = '2020-01-01'
end_date = '2022-04-21'
macro_params = {"SDate": start_date, "EDate": end_date}

df, err = get_data(["US10YT=RR", "BRT-"], None, params)

[INFO] Number of dates: 589
[INFO] Number of tickers at once: 1
[INFO] Max time window length: 416
Starting for loop


Ticker:   0%|          | 0/2 [00:00<?, ?it/s]

In [133]:
df

,HIGH,CLOSE,LOW,OPEN,ticker,VALUE
Date,,,,,,
2020-01-02,1.946,1.882,1.851,1.919,US10YT=RR,<NA>
2020-01-03,1.844,1.788,1.786,1.842,US10YT=RR,<NA>
2020-01-06,1.816,1.811,1.76,1.769,US10YT=RR,<NA>
2020-01-07,1.828,1.825,1.795,1.813,US10YT=RR,<NA>
2020-01-08,1.877,1.874,1.705,1.767,US10YT=RR,<NA>
...,...,...,...,...,...,...
2022-04-29,<NA>,107.28,<NA>,<NA>,BRT-,107.28
2022-05-03,<NA>,105.79,<NA>,<NA>,BRT-,105.79
2022-05-04,<NA>,108.74,<NA>,<NA>,BRT-,108.74


In [167]:
df, err = get_data(global_oil.index[:3], profits, ek_params)

number_of_timepoints 9
number_of_tickers_at_once 500


  0%|          | 0/1 [00:00<?, ?it/s]